## Covariance Analysis

Based on the computation of the two-point covariance, we will compute three properties:
 - Slope of the Covariance at the origin $\frac{dS^{(1)}_2(r)}{dr}|_{r=0}$
 - The specific surface area $S_V$
 - Chord length for each phase $l^{(i)}_C$

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
import json

In [38]:
#strings to output and input locations
beadpack_dic = {
    "out_direc": "../../../analysis/covariance/beadpack/",
    "seed_min": 43,
    "seed_max": 64
}

berea_dic = {
    "out_direc": "../../../analysis/covariance/berea/",
    "seed_min": 43,
    "seed_max": 64
}

ketton_dic = {
    "out_direc": "../../../analysis/covariance/ketton/",
    "seed_min": 43,
    "seed_max": 64
}
data_dic = ketton_dic
out_direc = data_dic["out_direc"]

## Data Loading 

We load data using pandas from the given directory of the covariances.

In [39]:
orig_cov_pph = pd.read_csv(out_direc+"orig_pph.csv")
orig_cov_gph = pd.read_csv(out_direc+"orig_gph.csv")

We now compute the slope at the origin of the radial averaged covariance to evaluate the specific surface area.

$$S_V = -4 \frac{dS^{(1)}_2(r)}{dr}|_{r=0}$$

We do this by fittin a straight line at the origin and fixing the intercept at $S^{(1)}_2(0)=\phi$
Therefore the equation we are fitting is:

$$y = ax + \phi$$

In [40]:
def radial_average(cov):
    avg = np.mean(cov, axis=0)
    return avg


def straight_line_at_origin(porosity):
    def func(x, a):
        return a * x + porosity
    return func

In [41]:
original_average_pph = radial_average(orig_cov_pph.values.T)
original_average_gph = radial_average(orig_cov_gph.values.T)

In [42]:
N = 5
slope_pph, slope_pph_cov = curve_fit(straight_line_at_origin(original_average_pph[0]), range(0, N), original_average_pph[0:N])
slope_gph, slope_gph_cov = curve_fit(straight_line_at_origin(original_average_gph[0]), range(0, N), original_average_gph[0:N])
print slope_pph, slope_gph

specific_surface_orig = -4*slope_pph
print specific_surface_orig

[-0.01369403] [-0.01367592]
[ 0.05477614]


Finally we estimate the chord length of both phases by computing:

$$l^{(i)}_C=-\frac{\phi^{(i)}}{\frac{dS^{(i)}_2(r)}{dr}|_{r=0}}$$

In [43]:
chord_length_pph = -original_average_pph[0]/slope_pph
chord_length_gph = -original_average_gph[0]/slope_gph
print chord_length_pph, chord_length_gph

[ 9.26403769] [ 63.84489306]


In [28]:
orig_data = {
    "slope_gph": float(slope_gph), "slope_pph": float(slope_pph), 
    "specific_surface": float(specific_surface_orig), 
    "chord_length_pph": float(chord_length_pph), "chord_length_gph":float(chord_length_gph)}

In [29]:
covariance_values = {}
covariance_values["orig"] = orig_data

## Synthetic Samples Computation

We now perform the same computation for the synthetic samples

In [30]:
for i in range(data_dic["seed_min"], data_dic["seed_max"]):
    cov_pph = pd.read_csv(out_direc+"S_"+str(i)+"_pph.csv")
    cov_gph = pd.read_csv(out_direc+"S_"+str(i)+"_gph.csv")
    
    average_pph = radial_average(cov_pph.values.T)
    average_gph = radial_average(cov_gph.values.T)
    
    slope_pph, slope_pph_cov = curve_fit(straight_line_at_origin(average_pph[0]), range(0, N), average_pph[0:N])
    slope_gph, slope_gph_cov = curve_fit(straight_line_at_origin(average_gph[0]), range(0, N), average_gph[0:N])

    
    specific_surface = -4*slope_pph

    
    chord_length_pph = -average_pph[0]/slope_pph
    chord_length_gph = -average_gph[0]/slope_gph

    data = {
    "slope_gph": float(slope_gph), "slope_pph": float(slope_pph), 
    "specific_surface": float(specific_surface), 
    "chord_length_pph": float(chord_length_pph), "chord_length_gph":float(chord_length_gph)}
    covariance_values["S_"+str(i)] = data

## Data Output to JSON
And finally we dump everything to a json file that let's us use this data in future graphs and analysis.

In [31]:
with open(out_direc+"covariance_data.json", "w") as f:
    json.dump(covariance_values, f)